# Pystac-client Python Library
* [Medium post](https://medium.com/rotten-grapes/download-sentinel-data-within-seconds-in-python-8cc9a8c3e23c) - pystac-client & odc-stac
* [Pystac-client documentation](https://pystac-client.readthedocs.io/en/latest/)

In [3]:
from pystac_client import Client
from odc.stac import load
import odc.geo
import pathlib

In [4]:
data_path = pathlib.Path.cwd() / ".." / "data"
(data_path / "rasters").mkdir(parents=True, exist_ok=True)

Link to available STAC catalogues: [https://stacindex.org/catalogs#/](https://stacindex.org/catalogs#/)
* Copernicus - "https://catalogue.dataspace.copernicus.eu/stac"
* Earth Search - "https://earth-search.aws.element84.com/v1"

In [5]:
# use publically available stac link such as
client = Client.open("https://earth-search.aws.element84.com/v1") 

# ID of the collection
collection = "sentinel-2-l2a"

# Geometry of AOI
geometry = {
    "coordinates": [
        [
            [74.66218437999487, 19.46556170905807],
            [74.6629598736763, 19.466339343697722],
            [74.6640371158719, 19.4667885366414],
            [74.66395296156406, 19.46614872872264],
            [74.66376889497042, 19.466150941501425],
            [74.66369077563286, 19.46577508478787],
            [74.6635865047574, 19.465278788212864],
            [74.66282073408365, 19.46540270444271],
            [74.66218437999487, 19.46556170905807],
        ]
    ],
    "type": "Polygon",
}

In [9]:
# Date range
date_YYMM = "2023-01"
filters = {
    "eo:cloud_cover":{"lt":0.2},
    "s2:vegetation_percentage": {"gt": 25}
}
# run pystac client search to see available dataset
search = client.search(collections=[collection], intersects=geometry , query=filters ,datetime=date_YYMM)
# spit out data as GeoJSON dictionary
print(search.item_collection_as_dict())
# loop through each item
for item in search.items_as_dicts():
    print(item)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'stac_version': '1.0.0', 'id': 'S2A_43QDB_20230109_0_L2A', 'properties': {'created': '2023-01-09T13:41:20.389Z', 'platform': 'sentinel-2a', 'constellation': 'sentinel-2', 'instruments': ['msi'], 'eo:cloud_cover': 0.000664, 'proj:epsg': 32643, 'mgrs:utm_zone': 43, 'mgrs:latitude_band': 'Q', 'mgrs:grid_square': 'DB', 'grid:code': 'MGRS-43QDB', 'view:sun_azimuth': 152.409883529928, 'view:sun_elevation': 43.5651505036477, 's2:degraded_msi_data_percentage': 0.0001, 's2:nodata_pixel_percentage': 0, 's2:saturated_defective_pixel_percentage': 0, 's2:dark_features_percentage': 0, 's2:cloud_shadow_percentage': 0.006662, 's2:vegetation_percentage': 32.694611, 's2:not_vegetated_percentage': 66.053742, 's2:water_percentage': 1.243841, 's2:unclassified_percentage': 0.000484, 's2:medium_proba_clouds_percentage': 1e-05, 's2:high_proba_clouds_percentage': 7e-06, 's2:thin_cirrus_percentage': 0.000647, 's2:snow_ice_percentage': 0, 's2:product

In [11]:
#load the data in xarray format
data = load(search.items() ,geopolygon=geometry,groupby="solar_day", chunks={})
data

<xarray.Dataset> Size: 24kB
Dimensions:       (y: 18, x: 20, time: 1)
Coordinates:
  * y             (y) float64 144B 2.153e+06 2.153e+06 ... 2.152e+06 2.152e+06
  * x             (x) float64 160B 4.645e+05 4.646e+05 ... 4.647e+05 4.647e+05
    spatial_ref   int32 4B 32643
  * time          (time) datetime64[ns] 8B 2023-01-09T05:43:37.058000
Data variables: (12/32)
    aot           (time, y, x) uint16 720B dask.array<chunksize=(1, 18, 20), meta=np.ndarray>
    blue          (time, y, x) uint16 720B dask.array<chunksize=(1, 18, 20), meta=np.ndarray>
    coastal       (time, y, x) uint16 720B dask.array<chunksize=(1, 18, 20), meta=np.ndarray>
    green         (time, y, x) uint16 720B dask.array<chunksize=(1, 18, 20), meta=np.ndarray>
    nir           (time, y, x) uint16 720B dask.array<chunksize=(1, 18, 20), meta=np.ndarray>
    nir08         (time, y, x) uint16 720B dask.array<chunksize=(1, 18, 20), meta=np.ndarray>
    ...            ...
    rededge3-jp2  (time, y, x) uint16 720B dask.array<chunksize=(1, 18, 20), meta=np.ndarray>
    scl-jp2       (time, y, x) uint8 360B dask.array<chunksize=(1, 18, 20), meta=np.ndarray>
    swir16-jp2    (time, y, x) uint16 720B dask.array<chunksize=(1, 18, 20), meta=np.ndarray>
    swir22-jp2    (time, y, x) uint16 720B dask.array<chunksize=(1, 18, 20), meta=np.ndarray>
    visual-jp2    (time, y, x) float32 1kB dask.array<chunksize=(1, 18, 20), meta=np.ndarray>
    wvp-jp2       (time, y, x) uint16 720B dask.array<chunksize=(1, 18, 20), meta=np.ndarray>

In [12]:
# create the index without considering scale or offset
ndvi = (data.nir - data.red) / (data.nir + data.red)

In [13]:
# export data as tiff
odc.geo.xr.write_cog(ndvi,fname=data_path / "rasters" / 'ndvi.tiff',  overwrite=True)

WindowsPath('c:/Local/repos/kelp-dashboard-demo/notebooks/../data/rasters/ndvi.tiff')